In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
out = 'output/1601606884'

In [4]:
import tensorflow_text  
import tensorflow as tf

tf.compat.v1.reset_default_graph()
config = tf.ConfigProto()
config.allow_soft_placement = True
sess = tf.InteractiveSession(config = config)
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], out)
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1601606884/variables/variables


In [5]:
import tensorflow as tf
import sentencepiece as spm

vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s)
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [6]:
import json

# !wget https://f000.backblazeb2.com/file/malay-dataset/testset/paraphrase-set.json

with open('../summary/paraphrase-set.json') as fopen:
    test = json.load(fopen)
test.keys()

dict_keys(['before', 'test_before', 'after', 'test_after'])

In [7]:
len(test['test_before'])

1000

In [8]:
from tqdm import tqdm

results = []
batch_size = 10

for i in tqdm(range(0, len(test['test_before']), batch_size)):
    r = pred(test['test_before'][i: i + batch_size])
    results.extend([i.decode() for i in r])

100%|██████████| 100/100 [16:15<00:00,  9.75s/it]


In [9]:
from tensor2tensor.utils import bleu_hook

In [10]:
bleu_hook.compute_bleu(reference_corpus = test['test_after'], 
                       translation_corpus = results)

0.81801796